In [86]:
import requests 
import re
import xlrd
import glob
import csv
session = requests.Session()
textfiles=glob.glob('./*.xlsx')
for u in range(len(textfiles)):
    sheet = xlrd.open_workbook(textfiles[u]).sheet_by_index(0)
    samplenum=len(sheet.col_values(0))
    with open(str(textfiles[u])+' Lipid Droplets.csv', 'w', newline='') as file:
        csv.writer(file).writerow(['Protein.Ids','Subcellular Location','Gene','Protein.Description','# of protein ids per sample','Peptides 3_Reg_IP','Peptides KO_1','Peptides KO_2','Peptides KO_3','Peptides WT_1','Peptides WT_2','Peptides WT_3'])
        for r in range(1,samplenum):
            rowvalues=sheet.row_values(r)
            splitlist=rowvalues[0].split(";")
            print('Analyzing '+str(r)+' of '+str(samplenum-1))
            main=[session.get('https://www.uniprot.org/uniprot/'+splitlist[s]).text for s in range(len(splitlist))]
            lipid=0
            for o in reversed(range(len(main))):
                if 'Endosome' in main[o]:
                    lipid=1
                if 'Golgi apparatus' in main[o]:
                    lipid=1
                if 'Cell membrane' in main[o]:
                    lipid=1
                if 'var isObsolete = true' in main[o]:
                    main.pop(o)
            if lipid==1:
                geneproteinlist=[re.findall(r'<head><title>.+?\s-\s.*?<',main[u]) for u in range(len(main))]
                gene=[geneproteinlist[t][0][13:geneproteinlist[t][0].index(' ')] for t in range(len(geneproteinlist))]
                protein=[]
                for o in range(len(geneproteinlist)):
                    if ' - ' in geneproteinlist[o][0][15+len(gene[o]):]:
                        startstring=geneproteinlist[o][0].index(' - ')
                        if ' - ' in geneproteinlist[o][0][startstring+3:]:
                            endstring=geneproteinlist[o][0][startstring+3:].index(' - ')
                            protein.append(geneproteinlist[o][0][startstring+3:(startstring+endstring+3)])
                        if ' - ' not in geneproteinlist[o][0][startstring+3:]:
                            astring=geneproteinlist[5][0][13:]
                            protein.append(astring[:(astring.index(' - '))])
                    if ' - ' not in geneproteinlist[o][0][15+len(gene[o]):]:
                        protein=gene
                location0=[re.findall("SL-\d{4}.*?<",main[u]) for u in range(len(main))]
                mark=0
                for p in range(len(location0)):
                    locationp=[(location0[p][y][location0[p][y].index('>')+1:location0[p][y].index('<')]) for y in range(len(location0[p]))]
                    if locationp!=[]:
                        stringlocationp=str(locationp)
                        if 'Endosome' in stringlocationp:
                            csv.writer(file).writerow([splitlist[p],locationp,gene[p],protein[p],len(splitlist),rowvalues[14],rowvalues[12],rowvalues[13],rowvalues[14],rowvalues[15],rowvalues[16],rowvalues[17], 'Endosome'])
                        if 'Golgi apparatus' in stringlocationp:
                            csv.writer(file).writerow([splitlist[p],locationp,gene[p],protein[p],len(splitlist),rowvalues[14],rowvalues[12],rowvalues[13],rowvalues[14],rowvalues[15],rowvalues[16],rowvalues[17], 'Golgi'])
                        if 'Cell membrane' in stringlocationp:
                            csv.writer(file).writerow([splitlist[p],locationp,gene[p],protein[p],len(splitlist),rowvalues[14],rowvalues[12],rowvalues[13],rowvalues[14],rowvalues[15],rowvalues[16],rowvalues[17], 'Membrane'])
                    if locationp==[]:
                        mark=1
                if mark==1:
                    location1=[re.findall(r'<h6>.+?</h6><ul..',main[u]) for u in range(len(main))]
                    location=[]
                    for d in range(len(location1)):
                        sublist=[]
                        for r in reversed(range(len(location1[d]))):
                            if 'ul c' in location1[d][r]:
                                subcell=location1[d][r][4:len(location1[d][r])-10]
                                if subcell!='Other locations':
                                    sublist.append(subcell)                     
                        location.append(sublist)
                    for x in range(len(location)):
                        flocation=location[x]
                        if location[x]==[]:
                            subcelllocation=[re.findall(r'subcellLocations.*',main[u]) for u in range(len(main))]
                            if [] not in subcelllocation:
                                location2=[re.findall(r'Display-GO-Term.*?<',subcelllocation[0][g]) for g in range(len(subcelllocation[0]))][0]
                            if [] in subcelllocation:
                                location2=[['>No Data<'] if x==[] else x for x in subcelllocation][0]
                            flocation=[(location2[y][location2[y].index('>')+1:location2[y].index('<')]) for y in range(len(location2))]
                        stringflocation=str(flocation)
                        if 'Endosome' in stringflocation:
                            csv.writer(file).writerow([splitlist[x],flocation,gene[x],protein[x],len(splitlist),rowvalues[14],rowvalues[12],rowvalues[13],rowvalues[14],rowvalues[15],rowvalues[16],rowvalues[17], 'Endosome'])
                        if 'Golgi apparatus' in stringflocation:
                            csv.writer(file).writerow([splitlist[x],flocation,gene[x],protein[x],len(splitlist),rowvalues[14],rowvalues[12],rowvalues[13],rowvalues[14],rowvalues[15],rowvalues[16],rowvalues[17], 'Golgi'])
                        if 'Cell membrane' in stringflocation:
                            csv.writer(file).writerow([splitlist[x],flocation,gene[x],protein[x],len(splitlist),rowvalues[14],rowvalues[12],rowvalues[13],rowvalues[14],rowvalues[15],rowvalues[16],rowvalues[17], 'Membrane'])
print('Finished processing the files')

Analyzing 1 of 1322
Analyzing 2 of 1322
Analyzing 3 of 1322
Analyzing 4 of 1322
Analyzing 5 of 1322
Analyzing 6 of 1322
Analyzing 7 of 1322
Analyzing 8 of 1322
Analyzing 9 of 1322
Analyzing 10 of 1322
Analyzing 11 of 1322
Analyzing 12 of 1322
Analyzing 13 of 1322
Analyzing 14 of 1322
Analyzing 15 of 1322
Analyzing 16 of 1322
Analyzing 17 of 1322
Analyzing 18 of 1322
Analyzing 19 of 1322
Analyzing 20 of 1322
Analyzing 21 of 1322
Analyzing 22 of 1322
Analyzing 23 of 1322
Analyzing 24 of 1322
Analyzing 25 of 1322
Analyzing 26 of 1322
Analyzing 27 of 1322
Analyzing 28 of 1322
Analyzing 29 of 1322
Analyzing 30 of 1322
Analyzing 31 of 1322
Analyzing 32 of 1322
Analyzing 33 of 1322
Analyzing 34 of 1322
Analyzing 35 of 1322
Analyzing 36 of 1322
Analyzing 37 of 1322
Analyzing 38 of 1322
Analyzing 39 of 1322
Analyzing 40 of 1322
Analyzing 41 of 1322
Analyzing 42 of 1322
Analyzing 43 of 1322
Analyzing 44 of 1322
Analyzing 45 of 1322
Analyzing 46 of 1322
Analyzing 47 of 1322
Analyzing 48 of 1322
A

Analyzing 379 of 1322
Analyzing 380 of 1322
Analyzing 381 of 1322
Analyzing 382 of 1322
Analyzing 383 of 1322
Analyzing 384 of 1322
Analyzing 385 of 1322
Analyzing 386 of 1322
Analyzing 387 of 1322
Analyzing 388 of 1322
Analyzing 389 of 1322
Analyzing 390 of 1322
Analyzing 391 of 1322
Analyzing 392 of 1322
Analyzing 393 of 1322
Analyzing 394 of 1322
Analyzing 395 of 1322
Analyzing 396 of 1322
Analyzing 397 of 1322
Analyzing 398 of 1322
Analyzing 399 of 1322
Analyzing 400 of 1322
Analyzing 401 of 1322
Analyzing 402 of 1322
Analyzing 403 of 1322
Analyzing 404 of 1322
Analyzing 405 of 1322
Analyzing 406 of 1322
Analyzing 407 of 1322
Analyzing 408 of 1322
Analyzing 409 of 1322
Analyzing 410 of 1322
Analyzing 411 of 1322
Analyzing 412 of 1322
Analyzing 413 of 1322
Analyzing 414 of 1322
Analyzing 415 of 1322
Analyzing 416 of 1322
Analyzing 417 of 1322
Analyzing 418 of 1322
Analyzing 419 of 1322
Analyzing 420 of 1322
Analyzing 421 of 1322
Analyzing 422 of 1322
Analyzing 423 of 1322
Analyzing 

Analyzing 752 of 1322
Analyzing 753 of 1322
Analyzing 754 of 1322
Analyzing 755 of 1322
Analyzing 756 of 1322
Analyzing 757 of 1322
Analyzing 758 of 1322
Analyzing 759 of 1322
Analyzing 760 of 1322
Analyzing 761 of 1322
Analyzing 762 of 1322
Analyzing 763 of 1322
Analyzing 764 of 1322
Analyzing 765 of 1322
Analyzing 766 of 1322
Analyzing 767 of 1322
Analyzing 768 of 1322
Analyzing 769 of 1322
Analyzing 770 of 1322
Analyzing 771 of 1322
Analyzing 772 of 1322
Analyzing 773 of 1322
Analyzing 774 of 1322
Analyzing 775 of 1322
Analyzing 776 of 1322
Analyzing 777 of 1322
Analyzing 778 of 1322
Analyzing 779 of 1322
Analyzing 780 of 1322
Analyzing 781 of 1322
Analyzing 782 of 1322
Analyzing 783 of 1322
Analyzing 784 of 1322
Analyzing 785 of 1322
Analyzing 786 of 1322
Analyzing 787 of 1322
Analyzing 788 of 1322
Analyzing 789 of 1322
Analyzing 790 of 1322
Analyzing 791 of 1322
Analyzing 792 of 1322
Analyzing 793 of 1322
Analyzing 794 of 1322
Analyzing 795 of 1322
Analyzing 796 of 1322
Analyzing 

Analyzing 1120 of 1322
Analyzing 1121 of 1322
Analyzing 1122 of 1322
Analyzing 1123 of 1322
Analyzing 1124 of 1322
Analyzing 1125 of 1322
Analyzing 1126 of 1322
Analyzing 1127 of 1322
Analyzing 1128 of 1322
Analyzing 1129 of 1322
Analyzing 1130 of 1322
Analyzing 1131 of 1322
Analyzing 1132 of 1322
Analyzing 1133 of 1322
Analyzing 1134 of 1322
Analyzing 1135 of 1322
Analyzing 1136 of 1322
Analyzing 1137 of 1322
Analyzing 1138 of 1322
Analyzing 1139 of 1322
Analyzing 1140 of 1322
Analyzing 1141 of 1322
Analyzing 1142 of 1322
Analyzing 1143 of 1322
Analyzing 1144 of 1322
Analyzing 1145 of 1322
Analyzing 1146 of 1322
Analyzing 1147 of 1322
Analyzing 1148 of 1322
Analyzing 1149 of 1322
Analyzing 1150 of 1322
Analyzing 1151 of 1322
Analyzing 1152 of 1322
Analyzing 1153 of 1322
Analyzing 1154 of 1322
Analyzing 1155 of 1322
Analyzing 1156 of 1322
Analyzing 1157 of 1322
Analyzing 1158 of 1322
Analyzing 1159 of 1322
Analyzing 1160 of 1322
Analyzing 1161 of 1322
Analyzing 1162 of 1322
Analyzing 1

Analyzing 167 of 2485
Analyzing 168 of 2485
Analyzing 169 of 2485
Analyzing 170 of 2485
Analyzing 171 of 2485
Analyzing 172 of 2485
Analyzing 173 of 2485
Analyzing 174 of 2485
Analyzing 175 of 2485
Analyzing 176 of 2485
Analyzing 177 of 2485
Analyzing 178 of 2485
Analyzing 179 of 2485
Analyzing 180 of 2485
Analyzing 181 of 2485
Analyzing 182 of 2485
Analyzing 183 of 2485
Analyzing 184 of 2485
Analyzing 185 of 2485
Analyzing 186 of 2485
Analyzing 187 of 2485
Analyzing 188 of 2485
Analyzing 189 of 2485
Analyzing 190 of 2485
Analyzing 191 of 2485
Analyzing 192 of 2485
Analyzing 193 of 2485
Analyzing 194 of 2485
Analyzing 195 of 2485
Analyzing 196 of 2485
Analyzing 197 of 2485
Analyzing 198 of 2485
Analyzing 199 of 2485
Analyzing 200 of 2485
Analyzing 201 of 2485
Analyzing 202 of 2485
Analyzing 203 of 2485
Analyzing 204 of 2485
Analyzing 205 of 2485
Analyzing 206 of 2485
Analyzing 207 of 2485
Analyzing 208 of 2485
Analyzing 209 of 2485
Analyzing 210 of 2485
Analyzing 211 of 2485
Analyzing 

...

Analyzing 914 of 2485
Analyzing 915 of 2485
Analyzing 916 of 2485
Analyzing 917 of 2485
Analyzing 918 of 2485
Analyzing 919 of 2485
Analyzing 920 of 2485
Analyzing 921 of 2485
Analyzing 922 of 2485
Analyzing 923 of 2485
Analyzing 924 of 2485
Analyzing 925 of 2485
Analyzing 926 of 2485
Analyzing 927 of 2485
Analyzing 928 of 2485
Analyzing 929 of 2485
Analyzing 930 of 2485
Analyzing 931 of 2485
Analyzing 932 of 2485
Analyzing 933 of 2485
Analyzing 934 of 2485
Analyzing 935 of 2485
Analyzing 936 of 2485
Analyzing 937 of 2485
Analyzing 938 of 2485
Analyzing 939 of 2485
Analyzing 940 of 2485
Analyzing 941 of 2485
Analyzing 942 of 2485
Analyzing 943 of 2485
Analyzing 944 of 2485
Analyzing 945 of 2485
Analyzing 946 of 2485
Analyzing 947 of 2485
Analyzing 948 of 2485
Analyzing 949 of 2485
Analyzing 950 of 2485
Analyzing 951 of 2485
Analyzing 952 of 2485
Analyzing 953 of 2485
Analyzing 954 of 2485
Analyzing 955 of 2485
Analyzing 956 of 2485
Analyzing 957 of 2485
Analyzing 958 of 2485
Analyzing 

Analyzing 1274 of 2485
Analyzing 1275 of 2485
Analyzing 1276 of 2485
Analyzing 1277 of 2485
Analyzing 1278 of 2485
Analyzing 1279 of 2485
Analyzing 1280 of 2485
Analyzing 1281 of 2485
Analyzing 1282 of 2485
Analyzing 1283 of 2485
Analyzing 1284 of 2485
Analyzing 1285 of 2485
Analyzing 1286 of 2485
Analyzing 1287 of 2485
Analyzing 1288 of 2485
Analyzing 1289 of 2485
Analyzing 1290 of 2485
Analyzing 1291 of 2485
Analyzing 1292 of 2485
Analyzing 1293 of 2485
Analyzing 1294 of 2485
Analyzing 1295 of 2485
Analyzing 1296 of 2485
Analyzing 1297 of 2485
Analyzing 1298 of 2485
Analyzing 1299 of 2485
Analyzing 1300 of 2485
Analyzing 1301 of 2485
Analyzing 1302 of 2485
Analyzing 1303 of 2485
Analyzing 1304 of 2485
Analyzing 1305 of 2485
Analyzing 1306 of 2485
Analyzing 1307 of 2485
Analyzing 1308 of 2485
Analyzing 1309 of 2485
Analyzing 1310 of 2485
Analyzing 1311 of 2485
Analyzing 1312 of 2485
Analyzing 1313 of 2485
Analyzing 1314 of 2485
Analyzing 1315 of 2485
Analyzing 1316 of 2485
Analyzing 1

Analyzing 1631 of 2485
Analyzing 1632 of 2485
Analyzing 1633 of 2485
Analyzing 1634 of 2485
Analyzing 1635 of 2485
Analyzing 1636 of 2485
Analyzing 1637 of 2485
Analyzing 1638 of 2485
Analyzing 1639 of 2485
Analyzing 1640 of 2485
Analyzing 1641 of 2485
Analyzing 1642 of 2485
Analyzing 1643 of 2485
Analyzing 1644 of 2485
Analyzing 1645 of 2485
Analyzing 1646 of 2485
Analyzing 1647 of 2485
Analyzing 1648 of 2485
Analyzing 1649 of 2485
Analyzing 1650 of 2485
Analyzing 1651 of 2485
Analyzing 1652 of 2485
Analyzing 1653 of 2485
Analyzing 1654 of 2485
Analyzing 1655 of 2485
Analyzing 1656 of 2485
Analyzing 1657 of 2485
Analyzing 1658 of 2485
Analyzing 1659 of 2485
Analyzing 1660 of 2485
Analyzing 1661 of 2485
Analyzing 1662 of 2485
Analyzing 1663 of 2485
Analyzing 1664 of 2485
Analyzing 1665 of 2485
Analyzing 1666 of 2485
Analyzing 1667 of 2485
Analyzing 1668 of 2485
Analyzing 1669 of 2485
Analyzing 1670 of 2485
Analyzing 1671 of 2485
Analyzing 1672 of 2485
Analyzing 1673 of 2485
Analyzing 1

Analyzing 1989 of 2485
Analyzing 1990 of 2485
Analyzing 1991 of 2485
Analyzing 1992 of 2485
Analyzing 1993 of 2485
Analyzing 1994 of 2485
Analyzing 1995 of 2485
Analyzing 1996 of 2485
Analyzing 1997 of 2485
Analyzing 1998 of 2485
Analyzing 1999 of 2485
Analyzing 2000 of 2485
Analyzing 2001 of 2485
Analyzing 2002 of 2485
Analyzing 2003 of 2485
Analyzing 2004 of 2485
Analyzing 2005 of 2485
Analyzing 2006 of 2485
Analyzing 2007 of 2485
Analyzing 2008 of 2485
Analyzing 2009 of 2485
Analyzing 2010 of 2485
Analyzing 2011 of 2485
Analyzing 2012 of 2485
Analyzing 2013 of 2485
Analyzing 2014 of 2485
Analyzing 2015 of 2485
Analyzing 2016 of 2485
Analyzing 2017 of 2485
Analyzing 2018 of 2485
Analyzing 2019 of 2485
Analyzing 2020 of 2485
Analyzing 2021 of 2485
Analyzing 2022 of 2485
Analyzing 2023 of 2485
Analyzing 2024 of 2485
Analyzing 2025 of 2485
Analyzing 2026 of 2485
Analyzing 2027 of 2485
Analyzing 2028 of 2485
Analyzing 2029 of 2485
Analyzing 2030 of 2485
Analyzing 2031 of 2485
Analyzing 2

Analyzing 2346 of 2485
Analyzing 2347 of 2485
Analyzing 2348 of 2485
Analyzing 2349 of 2485
Analyzing 2350 of 2485
Analyzing 2351 of 2485
Analyzing 2352 of 2485
Analyzing 2353 of 2485
Analyzing 2354 of 2485
Analyzing 2355 of 2485
Analyzing 2356 of 2485
Analyzing 2357 of 2485
Analyzing 2358 of 2485
Analyzing 2359 of 2485
Analyzing 2360 of 2485
Analyzing 2361 of 2485
Analyzing 2362 of 2485
Analyzing 2363 of 2485
Analyzing 2364 of 2485
Analyzing 2365 of 2485
Analyzing 2366 of 2485
Analyzing 2367 of 2485
Analyzing 2368 of 2485
Analyzing 2369 of 2485
Analyzing 2370 of 2485
Analyzing 2371 of 2485
Analyzing 2372 of 2485
Analyzing 2373 of 2485
Analyzing 2374 of 2485
Analyzing 2375 of 2485
Analyzing 2376 of 2485
Analyzing 2377 of 2485
Analyzing 2378 of 2485
Analyzing 2379 of 2485
Analyzing 2380 of 2485
Analyzing 2381 of 2485
Analyzing 2382 of 2485
Analyzing 2383 of 2485
Analyzing 2384 of 2485
Analyzing 2385 of 2485
Analyzing 2386 of 2485
Analyzing 2387 of 2485
Analyzing 2388 of 2485
Analyzing 2